In [1]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2

# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 

In [2]:
for v in world.get_actors().filter('*vehicle*'):
    v.destroy()
for s in world.get_actors().filter('*sensor*'):
    s.destroy()

In [3]:
# Spawn ego vehicle
vehicle_bp = bp_lib.find('vehicle.tesla.model3') 
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

# Move spectator behind vehicle to view
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

In [23]:
import importlib
import sys
from car.sensors import CarEquipment, base_sensor_specification, model3_base_offsets, base_sensor_types, base_callbacks, top_offsets, top_callbacks

In [5]:
 #import the module here, so that it can be reloaded.
importlib.reload(sys.modules['car.sensors'])
from car.sensors import CarEquipment, base_sensor_specification, model3_base_offsets, base_sensor_types, base_callbacks, top_offsets, top_callbacks

In [6]:
equipment = CarEquipment(world, vehicle)

In [7]:
equipment.build_sensors('base', base_sensor_types[0], base_sensor_specification, model3_base_offsets)

In [8]:
equipment.base_sensors['front'].is_listening

False

In [9]:
equipment.toggle_sensors('base', 'start', equipment.sem_callback, base_callbacks)

In [10]:
img = equipment.sensor_data['rear']

In [11]:
def layout_func(sensor_data):
    row1 = np.concatenate([sensor_data['front_left'], sensor_data['front'], sensor_data['front_right']], axis=1)
    row2 = np.concatenate([sensor_data['side_left'], sensor_data['rear'], sensor_data['side_right']], axis=1)
    tiled = np.concatenate([row1,row2], axis=0)
    return tiled

In [12]:
equipment.display_sensors('Base Segmentation', layout_func, equipment.sensor_data)

In [13]:
equipment.build_sensors('top', base_sensor_types[0], base_sensor_specification, top_offsets)

In [17]:
equipment.toggle_sensors('top', 'start', equipment.sem_callback, top_callbacks)

In [20]:
def top_layout(sensor_data):
    return sensor_data['hd1']

In [25]:
equipment.display_sensors('Base Segmentation', top_layout, equipment.sensor_data)